In [34]:
import glob
import numpy as np
import os
import re
import pandas as pd
import os.path
import xml.etree.ElementTree as etree
import csv
import getopt
import sys
import json
import copy
import tensorflow.keras.utils as TKU
from random import shuffle
from sklearn import utils
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
import keras
import keras.preprocessing.text as kpt
import nltk
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, LSTM, Input, Embedding
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, concatenate, Activation, Average
from keras.models import Model
from keras import optimizers
from keras.callbacks import CSVLogger
from nltk.tokenize import TreebankWordTokenizer
from keras.models import model_from_json

In [10]:
inputDir = 'C:\\Users\\morzm\\jup_txts\\csv'
outputDir = 'C:\\Users\\morzm\\jup_txts\\csv\\output'
modelDir = 'C:\\Users\\morzm\\jup_txts\\csv\\model'

In [11]:
xml_path = "C:\\Users\\morzm\\jup_txts\\csv\\en\\text\\"
label_path = "C:\\Users\\morzm\\jup_txts\\csv\\en\\truth.txt"
label_file = open(label_path)
model_path = os.path.join(modelDir, 'en_model.json')
dic_path = os.path.join(modelDir, 'en_dictionary.json')
twit_path = os.path.join(modelDir, 'twitter_en_data_set.csv')

In [12]:
with open(twit_path, 'w+', encoding='utf-8') as csvfile:
    fieldnames = ['text', 'gender']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for filename in os.listdir(xml_path):
        if not filename.endswith('.xml'): continue
        # ----reading file name
        find_file_name = os.path.splitext(filename)[0]
        # print(find_file_name)
        # ----finding author gender
        label_file = open(label_path)
        for row in label_file:
            if find_file_name in row:
                # print(find_file_name)
                # print(row.split(':::')[1])
                gender = row.split(':::')[1]

        # print(gender)

        # ----reading xml full path
        xml_fullname = os.path.join(xml_path, filename)
        tree = etree.parse(xml_fullname)
        root = tree.getroot()
        # ----reading xml file data
        for i in range(len(root[0])):
            writer.writerow({'text': str(root[0][i].text), 'gender': str(gender)})
csvfile.close()

In [13]:
def stem(word):
    regexp = r'^(.*?)(ing|ious|ment)?$'
    stem, suffix = re.findall(regexp, word)[0]
    return stem

In [14]:
def tokenize(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    # text = re_sub(r"#\S+", "hashtag")
    # text = re_sub(r"([!?.]){2,}", r" \1 ")
    # text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 ")
    # text = re_sub(r"([A-Z]){2,}", "allcaps")
    # text = re_sub(r'([\w!.,?();*\[\]":\”\“])([!.,?();*\[\]":\”\“])', r'\1 \2')
    # text = re_sub(r'([!.,?();*:\[\]":\”\“])([\w!.,?();*\[\]":\”\“])', r'\1 \2')
    # text = re_sub(r'(.)(<)', r'\1 \2')
    # text = re_sub(r'(>)(.)', r'\1 \2')
    # text = re_sub(r'[\'\`\’\‘]', r'')
    # text = re_sub(r'\\n', r' ')
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " ")
    # text = re_sub(r"#(\S+)", r" ") # replace #name with name
    text = re_sub(r"(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))", " em_positive ")  # Smile -- :), : ), :-), (:, ( :, (-:, :')
    text = re_sub(r"(:\s?D|:-D|x-?D|X-?D)", " em_positive ")  # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    text = re_sub(r"(<3|:\*)", " em_positive ")  # Love -- <3, :*
    text = re_sub(r"(;-?\)|;-?D|\(-?;)", " em_positive ")  # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    text = re_sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', " em_negative ")  # Sad -- :-(, : (, :(, ):, )-:
    text = re_sub(r'(:,\(|:\'\(|:"\()', " em_negative ")  # Cry -- :,(, :'(, :"(
    text = re_sub(r"(-|\')", "")  # remove &
    text = re_sub(r"/", " / ")
    text = re_sub(r"@[0-9]+-", " ")
    text = re_sub(r"@\w+", " ")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), " em_positive ")
    text = re_sub(r"{}{}p+".format(eyes, nose), " em_positive ")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), " em_negative ")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), " em_neutralface ")
    text = re_sub(r"<3", " heart ")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " ")
    text = re_sub(r'-', r' ')
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "  ")
    #     text = re_sub(r"([pls?s]){2,}", r"\1")
    #     text = re_sub(r"([plz?z]){2,}", r"\1")
    text = re_sub(r'\\n', r' ')
    text = re_sub(r" app ", " application ")
    text = re_sub(r"app", " application")
    text = re_sub(r" wil ", " will ")
    text = re_sub(r" im ", " i am ")
    text = re_sub(r" al ", " all ")
    # text = re_sub(r"<3","love")
    text = re_sub(r" sx ", " sex ")
    text = re_sub(r" u ", " you ")
    text = re_sub(r" r ", " are ")
    text = re_sub(r" y ", " why ")
    text = re_sub(r" Y ", " WHY ")
    text = re_sub(r"Y ", " WHY ")
    text = re_sub(r" hv ", " have ")
    text = re_sub(r" c ", " see ")
    text = re_sub(r" bcz ", " because ")
    text = re_sub(r" coz ", " because ")
    text = re_sub(r" v ", " we ")
    text = re_sub(r" ppl ", " people ")
    text = re_sub(r" pepl ", " people ")
    text = re_sub(r" r b i ", " rbi ")
    text = re_sub(r" R B I ", " RBI ")
    text = re_sub(r" R b i ", " rbi ")
    text = re_sub(r" R ", " ARE ")
    text = re_sub(r" hav ", " have ")
    text = re_sub(r"R ", " ARE ")
    text = re_sub(r" U ", " you ")
    text = re_sub(r" 👎 ", " em_negative ")
    text = re_sub(r"U ", " you ")
    text = re_sub(r" pls ", " please ")
    text = re_sub(r"Pls ", "Please ")
    text = re_sub(r"plz ", "please ")
    text = re_sub(r"Plz ", "Please ")
    text = re_sub(r"PLZ ", "Please ")
    text = re_sub(r"Pls", "Please ")
    text = re_sub(r"plz", "please ")
    text = re_sub(r"Plz", "Please ")
    text = re_sub(r"PLZ", "Please ")
    text = re_sub(r" thankz ", " thanks ")
    text = re_sub(r" thnx ", " thanks ")
    text = re_sub(r"fuck\w+ ", " fuck ")
    text = re_sub(r"f\*\* ", " fuck ")
    text = re_sub(r"\*\*\*k ", " fuck ")
    text = re_sub(r"F\*\* ", " fuck ")
    text = re_sub(r"mo\*\*\*\*\* ", " fucker ")
    text = re_sub(r"b\*\*\*\* ", " blody ")
    text = re_sub(r" mc ", " fucker ")
    text = re_sub(r" MC ", " fucker ")
    text = re_sub(r" wtf ", " fuck ")
    text = re_sub(r" ch\*\*\*ya ", " fucker ")
    text = re_sub(r" ch\*\*Tya ", " fucker ")
    text = re_sub(r" ch\*\*Tia ", " fucker ")
    text = re_sub(r" C\*\*\*yas ", " fucker ")
    text = re_sub(r"l\*\*\*\* ", "shit ")
    text = re_sub(r" A\*\*\*\*\*\*S", " ASSHOLES")
    text = re_sub(r" di\*\*\*\*s", " cker")
    text = re_sub(r" nd ", " and ")
    text = re_sub(r"Nd ", "and ")
    text = re_sub(r"([!?!]){2,}", r"! ")
    text = re_sub(r"([.?.]){2,}", r". ")
    text = re_sub(r"([*?*]){2,}", r"* ")
    text = re_sub(r"([,?,]){2,}", r", ")
    text = re_sub(r"ha?ha", r" em_positive ")
    # text = re_sub(r"([!]){2,}", r"! ")
    # text = re_sub(r"([.]){2,}", r". ")
    # text = re_sub(r"([*]){2,}", r"* ")
    # text = re_sub(r"([,]){2,}", r", ")
    # text = re_sub(r"\n\r", " ")
    text = re_sub(r"(ind[vs]pak)", " india versus pakistan ")
    text = re_sub(r"(pak[vs]ind)", " pakistan versus india ")
    text = re_sub(r"(indvsuae)", " india versus United Arab Emirates ")
    text = re_sub(r"[sS]hut[Dd]own[jnuJNU]", " shut down jnu ")
    # text = re_sub(r"ShutDownJNU", " shut down jnu ")
    # text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " number ")
    # text = re_sub(r"(.)\1\1+", r"\1") # remove funnnnny --> funny
    return text

In [15]:
FLAGS = re.MULTILINE | re.DOTALL

In [16]:
my_df = pd.read_csv(twit_path)
my_df['gender'] = my_df['gender'].map({'female\r\n': 0, 'male\r\n': 1})
my_df['text'] = my_df['text'].apply(lambda x: " ".join(stem(x) for x in x.split()))
stop = stopwords.words('english')
my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in x.split(" ") if x not in stop))
my_word_stop = ['the', 'in', 'of', 'is', 'a', 'to', 'an', 'be']
my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in x.split(" ") if x not in my_word_stop))
my_df['text'] = my_df['text'].apply(lambda x: " ".join(tokenize(x) for x in x.split(" ")))
# my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in text_processor.pre_process_doc(x) if x not in stop))
freq = pd.Series(' '.join(my_df['text']).split()).value_counts()[:50]
freq = list(freq.index)
my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [17]:
from keras.preprocessing.text import Tokenizer

train_x = my_df.text
train_y = my_df.gender

# create a new Tokenizer
max_num = 2000

tokenizer = Tokenizer(num_words=max_num)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

In [19]:
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
dic_path = "C:\\Users\\morzm\\OneDrive\\Asztali gép\\txts\\model\\en_dictionary.json"

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open(dic_path, 'w+') as dictionary_file:
    json.dump(dictionary, dictionary_file)

In [20]:
def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    temp_wordIndices = []
    for word in kpt.text_to_word_sequence(text):
        if word in dictionary:
            temp_wordIndices.append(dictionary[word])
    return temp_wordIndices

In [21]:
allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

In [22]:
# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices, dtype="object")

In [23]:
# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
train_y = np_utils.to_categorical(train_y, 2)

model = Sequential()

model.add(Dense(1024, activation='relu', input_shape=(max_num,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              2049024   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 2,574,850
Trainable params: 2,574,850
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
modelDir = "C:\\Users\\morzm\\OneDrive\\Asztali gép\\txts\\model\\"

In [24]:
filepath="sequencing_the_data_try_n_error.{epoch:02d}-{val_loss:.4f}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger(os.path.join(modelDir,'en_model_log.csv'), append=True, separator=';')

In [25]:
model.fit(train_x, train_y,
    batch_size=32,
    epochs=2,
    verbose=1,
    validation_split=0.1,
    shuffle=True,callbacks = [csv_logger])

Epoch 1/2
8438/8438 [==============================] - 81s 9ms/step - loss: 0.6702 - accuracy: 0.5865 - val_loss: 0.6654 - val_accuracy: 0.5887
Epoch 2/2
8438/8438 [==============================] - 77s 9ms/step - loss: 0.6484 - accuracy: 0.6093 - val_loss: 0.6707 - val_accuracy: 0.5850


In [18]:
print(my_df)

                                                     text  gender
0                              One watch … Available Feb.       1
1       There complete sets box batman minifigs. Find ...       1
2                        If (maybe ), come arrangement. 👍       1
3       heard numerous toyworlds numbering, despite wa...       1
4            Just heard Web Dev meetup Hutt. Spread word!       1
...                                                   ...     ...
299995  Hey! Is studio? Heres q NZH. Who said Yes! Pri...       0
299996  God lett asswipe post selfpity piece Mothers D...       0
299997  Next week #HottieRatings pinch #SilverFox stal...       0
299998  Story relevant cos timeline. say though. We co...       0
299999          #SilverFox rat system GO FOR LAUNCH! #OTD       0

[300000 rows x 2 columns]


In [26]:
model_json = model.to_json()

with open(model_path, 'w+') as json_file:
    json_file.write(model_json)

model.save_weights(os.path.join(modelDir, 'en_model.h5'))

print('saved model!')

saved model!


In [37]:
# read in your saved model structure
json_file = open(model_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
# and create a model from that
model = model_from_json(loaded_model_json)
# and weight your nodes with your saved values
model.load_weights("C:\\Users\\morzm\\jup_txts\\csv\\model\\en_model.h5")

In [38]:
# read in our saved dictionary
with open(dic_path, 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

In [39]:
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    no_word = 0
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
        else:
            # print("'%s' not in training corpus; ignoring." %(word))
            not_found_word_list.append(word)
            no_word = no_word + 1
    return wordIndices, no_word

In [40]:
test_xml_path = "C:\\Users\\morzm\\jup_txts\\csv\\test_xml\\text\\"
result_xml_path = "C:\\Users\\morzm\\jup_txts\\csv\\test_xml\\output\\"
model_path = "C:\\Users\\morzm\\jup_txts\csv\\model\\en_model.json"
dic_path = "C:\\Users\\morzm\\jup_txts\\csv\\model\\en_dictionary.json"
temp_twit = os.path.join(outputDir, 'temp_twit.csv')
model_weight = os.path.join(modelDir, 'en_model.h5')

In [43]:
# ----reading xml file data
for filename in os.listdir(test_xml_path):
    if not filename.endswith('.xml'): continue
    # ----reading file name
    author_id = os.path.splitext(filename)[0]
    # ----reading xml full path
    xml_fullname = os.path.join(test_xml_path, filename)
    tree = etree.parse(xml_fullname)
    root = tree.getroot()
    # ----reading xml file data and storing in temp csv
    with open(temp_twit, 'w+') as csvfile:
        fieldnames = ['text']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(len(root[0])):
            writer.writerow({'text': (root[0][i].text).encode('utf-8')})
    csvfile.close()
    # --reading temp csv file
    my_df = pd.read_csv(temp_twit)
    my_df['text'] = my_df['text'].apply(lambda x: " ".join(stem(x) for x in x.split()))
    stop = stopwords.words('english')
    my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in x.split(" ") if x not in stop))
    my_df['text'] = my_df['text'].apply(lambda x: " ".join(tokenize(x) for x in x.split(" ")))
    # my_df['text'] = my_df['text'].apply(lambda x: " ".join(x for x in text_processor.pre_process_doc(x) if x not in stop))
    # we're still going to use a Tokenizer here, but we don't need to fit it
    tokenizer = Tokenizer(num_words=max_num)
    # for human-friendly printing
    labels = ['female', 'male']
    # this utility makes sure that all the words in your input
    # are registered in the dictionary
    # before trying to turn them into a matrix.
    not_found_word_list = []
    male = 0
    female = 0
    for row in my_df.text:
        # okay here's the interactive part
        evalSentence = row
        # format your input for the neural net
        testArr, no_word = convert_text_to_index_array(evalSentence)
        input = tokenizer.sequences_to_matrix([testArr], mode='binary')
        # predict which bucket your input belongs in
        pred = model.predict(input)
        gender = labels[np.argmax(pred)]
        if gender == "male":
            male += 1
        else:
            female += 1
    if male >= female:
        gender = "male"
    else:
        gender = "female"
    text_to_write = """<author id='%s'\n\tlang='en'\n\tgender_txt='%s'\n/>""" % (author_id, gender)
    xml_result = os.path.join(result_xml_path, filename)
    fo = open(xml_result, "w+")
    fo.write(text_to_write)
    fo.close()

In [49]:
# Predicting a new instance

# Notice we have both positive and negative words here
sample_1 = """I hate that the dismal weather had me down for so long, 
when will it break! Ugh, when does happiness return? The sun is 
blinding and the puffy clouds are too thin. I can't wait for the weekend."""

# The first value is a "fake" class (this is the expected input)

model.predict(sample_1)

IndexError: list index out of range